In [1]:
from sklearn.linear_model import LinearRegression
import pandas as pd
import numpy as np
import datetime
import pickle

In [2]:
#generate dates
date2016 = datetime.date(2016,7,3)
date2017 = datetime.date(2017,4,19)
date2012 = datetime.date(2012,9,24)
date2014 = datetime.date(2014,11,16)
thirty = (29,31)
fifteen = (14,16)
forty = (39,42)
fifty = (49,54)
sixty = (59,62)


In [3]:
def diff_month(d1, d2):
    return (d1.year - d2.year) * 12 + d1.month - d2.month

diff_month(datetime.date.today(),date2017)

11

In [4]:
def make_data_string(st,user_id,interval,fdate = date2017):
    datapoints = diff_month(datetime.date.today(),fdate)
    for i in range(datapoints):
        rnd = np.random.randint(*interval)
        odate = date2017 + datetime.timedelta(i*rnd)
        st += f'{user_id},{odate}|'
    return st    

In [9]:
datastring = make_data_string("",1,thirty)

In [23]:
datastring = make_data_string(datastring,9,thirty,fdate=date2017)
datastring

'1,2017-04-19|1,2017-05-18|1,2017-06-16|1,2017-07-18|1,2017-08-17|1,2017-09-16|1,2017-10-10|1,2017-11-08|1,2017-12-07|1,2018-01-14|1,2018-02-03|2,2017-04-19|2,2017-06-19|2,2017-08-19|2,2017-10-13|2,2017-12-11|2,2018-02-13|2,2018-04-20|2,2018-06-13|2,2018-08-12|2,2018-10-20|2,2018-12-20|2,2019-01-28|2,2019-03-28|2,2019-06-08|2,2019-08-07|2,2019-10-06|2,2019-11-19|2,2020-02-20|2,2020-04-03|2,2020-06-02|3,2017-04-19|3,2017-05-30|3,2017-07-10|3,2017-08-17|3,2017-09-22|3,2017-10-31|3,2017-12-21|3,2018-01-31|3,2018-02-25|3,2018-04-23|3,2018-05-14|3,2018-06-22|3,2018-07-31|3,2018-09-08|3,2018-10-17|3,2018-11-25|3,2019-01-03|3,2019-02-11|3,2019-03-22|3,2019-05-19|4,2017-04-19|4,2017-05-03|4,2017-05-19|4,2017-05-31|4,2017-06-18|4,2017-07-03|4,2017-07-18|4,2017-08-02|4,2017-08-09|4,2017-09-01|4,2017-09-16|5,2017-04-19|5,2017-06-07|5,2017-07-26|5,2017-09-22|5,2017-11-01|5,2018-01-04|5,2018-02-25|5,2018-03-28|5,2018-05-16|5,2018-08-09|5,2018-10-01|5,2018-10-21|5,2019-01-03|5,2019-01-29|5,2019-03-2

In [24]:
len(datastring.split('|'))

257

In [27]:
with open('data.csv','w') as f:
    for i in datastring.split('|'):
        f.write(i+'\n')

In [5]:
df = pd.read_csv('data.csv',header=None,date_parser=True)
df.columns = ['user_id','order_date']
df.head()

,user_id,order_date
0,1,2017-04-19
1,1,2017-05-18
2,1,2017-06-16
3,1,2017-07-18
4,1,2017-08-17


In [6]:
df.groupby('user_id').count()

,order_date
user_id,
1,11
2,20
3,20
4,11
5,40
6,11
7,66
8,66
9,11


In [ ]:
sorted_df = df.sort_values(by=['user_id','order_date'])

In [21]:
sorted_df['order_date'] = pd.to_datetime(sorted_df['order_date'])

In [29]:
sorted_df[sorted_df['user_id'] == 1]['order_date'].dt.days

AttributeError: 'DatetimeProperties' object has no attribute 'days'

In [31]:
a = sorted_df[sorted_df['user_id'] == 1]['order_date'][:-1]

In [32]:
 b = sorted_df[sorted_df['user_id'] == 1]['order_date'][1:]

In [48]:
(b.values - a.values).astype('timedelta64[D]')

array([29, 29, 32, 30, 30, 24, 29, 29, 38, 20], dtype='timedelta64[D]')

In [26]:
 sorted_df[sorted_df['user_id'] == 1]['order_date'][1:] - sorted_df[sorted_df['user_id'] == 1]['order_date'][:-1]

0       NaT
1    0 days
2    0 days
3    0 days
4    0 days
5    0 days
6    0 days
7    0 days
8    0 days
9    0 days
10      NaT
Name: order_date, dtype: timedelta64[ns]

In [49]:
def get_delta_array(sorted_df,user_id):
    a = sorted_df[sorted_df['user_id'] == user_id]['order_date'][:-1]
    b = sorted_df[sorted_df['user_id'] == user_id]['order_date'][1:]
    target = (b.values - a.values).astype('timedelta64[D]')
    return target

In [108]:
get_delta_array(sorted_df,1).astype('int')

array([29, 29, 32, 30, 30, 24, 29, 29, 38, 20])

In [113]:
def create_learn_df(sorted_df):
    temp_df = pd.DataFrame(columns=sorted_df.columns)
    ss = []
    for user in sorted_df.user_id.unique():
        delta_array = get_delta_array(sorted_df,user).astype('int')
    return pd.concat(ss)

In [151]:
main_df = create_learn_df(sorted_df)

In [152]:
main_df['product'] = -100

In [184]:
main_df.loc[(main_df['product'] == -100)]

,delta,user_id,product


In [186]:
main_df.head()

,delta,user_id,product
0,29,1,1
1,29,1,1
2,32,1,2
3,30,1,2
4,30,1,2


In [187]:
main_df.shape

(247, 3)

In [194]:
pdtr = LinearRegression(normalize=False)
pdtr.fit(main_df[['user_id','product']][:-50],main_df['delta'][:-50])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [195]:
pdtr.score(main_df[['user_id','product']][-50:],main_df['delta'][-50:])

0.87210425956147286